In [19]:
import numpy as np
import pandas as pd

#inputs
notional = 100000000
coupon = 0.04
term = 36 
CDR = 0.02
CPR = 0.25
severity = 0.25
advance_rate = 0.95
facility_rate = 0.02


In [23]:
#initialisation

begin_col_balance = []
end_col_balance = []
sch_end_balance = []
sch_interest = []
sch_principal = [] 
defaults = []
recovery = []
net_loss = []
prepayment = []
principal_paid = []
cashflow = []
begin_fac_balance = []


monthly_default_rate = 1 - (1 - CDR)**(1/12)
monthly_prepayment_rate = 1 - (1 - CPR)**(1/12)

begin_col_balance.append(notional)
begin_fac_balance.append(advance_rate * notional)


def sched_principal(p, t, r):
    return p*r*((1+r)**t)/((1+r)**t-1)

for i in range(term):
    sch_interest.append(coupon * begin_col_balance[i])
    sch_principal.append(sched_principal(begin_col_balance[i],term-i,coupon)-sch_interest[i])
    sch_end_balance.append(begin_col_balance[i]-sch_principal[i])
    defaults.append(monthly_default_rate * sch_end_balance[i])
    recovery.append(defaults[i] * (1 - severity))
    net_loss.append(defaults[i] * severity)
    prepayment.append(monthly_prepayment_rate * sch_end_balance[i])
    end_col_balance.append(sch_end_balance[i] - defaults[i] - prepayment[i])
    begin_col_balance.append(end_col_balance[i])
    
    

In [24]:
data = {
    "Beginning collateral balance": begin_col_balance[0:term],
    "scheduled interest": sch_interest,
    "scheduled principal": sch_principal,
    "scheduled ending balance": sch_end_balance,
    "defaults": defaults,
    "recovery": recovery,
    "net loss": net_loss,
    "prepayment": prepayment,
    "ending collateral balance": end_col_balance
}

df = pd.DataFrame(data)
pd.set_option('display.float_format', '{:,.2f}'.format)

In [25]:
df


,Beginning collateral balance,scheduled interest,scheduled principal,scheduled ending balance,defaults,recovery,net loss,prepayment,ending collateral balance
0,"100,000,000.00","4,000,000.00","1,288,687.80","98,711,312.20","166,046.50","124,534.87","41,511.62","2,338,315.44","96,206,950.27"
1,"96,206,950.27","3,848,278.01","1,306,232.78","94,900,717.49","159,636.54","119,727.40","39,909.13","2,248,048.45","92,493,032.50"
2,"92,493,032.50","3,699,721.30","1,324,016.63","91,169,015.87","153,359.28","115,019.46","38,339.82","2,159,650.32","88,856,006.27"
3,"88,856,006.27","3,554,240.25","1,342,042.60","87,513,963.67","147,210.96","110,408.22","36,802.74","2,073,067.90","85,293,684.81"
4,"85,293,684.81","3,411,747.39","1,360,313.99","83,933,370.82","141,187.89","105,890.92","35,296.97","1,988,249.29","81,803,933.63"
5,"81,803,933.63","3,272,157.35","1,378,834.13","80,425,099.50","135,286.48","101,464.86","33,821.62","1,905,143.88","78,384,669.14"
6,"78,384,669.14","3,135,386.77","1,397,606.42","76,987,062.72","129,503.21","97,127.41","32,375.80","1,823,702.20","75,033,857.31"
7,"75,033,857.31","3,001,354.29","1,416,634.29","73,617,223.02","123,834.66","92,876.00","30,958.67","1,743,876.01","71,749,512.34"
8,"71,749,512.34","2,869,980.49","1,435,921.21","70,313,591.13","118,277.48","88,708.11","29,569.37","1,665,618.18","68,529,695.47"
9,"68,529,695.47","2,741,187.82","1,455,470.72","67,074,224.75","112,828.41","84,621.31","28,207.10","1,588,882.69","65,372,513.65"


In [26]:
df.to_excel("Amortizing loan calculator.xlsx")